In [1]:
import os
# 必须在导入 pipeline 之前设置
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HOME"] = "/root/autodl-tmp/models"  # 同时控制 model 和 tokenizer 缓存

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. 准备工作
checkpoint = "uer/roberta-base-finetuned-dianping-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# 2. 原始文本
text = "这家餐厅太难吃了，服务员态度还差！"

# 3. Step 1: Tokenize (预处理)
inputs = tokenizer(text, return_tensors="pt")
# inputs 包含 input_ids 和 attention_mask

# 4. Step 2: Model Inference (模型推理)
# 不需要计算梯度，节省内存
with torch.no_grad():
    outputs = model(**inputs)
    # 获取 Logits (未归一化的分数)
    logits = outputs.logits
    print(f"模型原始输出 (Logits): {logits}")

# 5. Step 3: Post-processing (后处理)
# 使用 Softmax 将 logits 转换为概率
probabilities = F.softmax(logits, dim=-1)
print(f"概率分布: {probabilities}")

# 获取概率最大的标签索引
predicted_id = torch.argmax(probabilities).item()
# 使用 model.config.id2label 查表得到人类可读标签
predicted_label = model.config.id2label[predicted_id]

print(f"最终结果: {predicted_label}")

2026-02-03 22:21:13.520470: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-03 22:21:13.541709: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770128473.567011    7860 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770128473.574406    7860 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770128473.595253    7860 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

模型原始输出 (Logits): tensor([[ 2.8590, -2.7449]])
概率分布: tensor([[0.9963, 0.0037]])
最终结果: negative (stars 1, 2 and 3)
